In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import torch
from datasets import load_dataset
from datasets import Dataset
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

In [3]:
import pandas as pd

csv_file_path = '/content/lyrics-dataset.csv'
df = pd.read_csv(csv_file_path)

result_list = []

for index, row in df.iterrows():
    article = f"English: {row['text']}"
    generated_song = f"Hinglish: {row['song']}"

    result_list.append([article, generated_song])

print(result_list[5])


['English: Pimpri-Chinchwad police have busted a racket of illegally refilling domestic liquefied petroleum gas (LPG) into commercial cylinders at a godown in Chikhali area on Tuesday evening.', 'Hinglish: (Verse 1)\nPimpri-Chinchwad police ne kiya kaam bada,\nRacket bust kiya, illegal game ka parda.\nChikhali area mein, ek godown ka scene,\nTuesday evening mein, hui raid ka dream.\n\n(Chorus)\nIllegally refilling, LPG ka chakkar,\nDomestic cylinders mein, commercial ka shor.\nPolice ne kiya, action ka jor,\nChikhali mein, hua racket ka ant.\n\n(Verse 2)\nGodown mein chhupa, yeh kala dhanda,\nLPG ki tankiyo se, bhara hai andaaz.\nPolice ne pakda, criminals ka group,\nIllegal refilling, tha yeh unka scoop.\n\n(Chorus)\nIllegally refilling, LPG ka chakkar,\nDomestic cylinders mein, commercial ka shor.\nPolice ne kiya, action ka jor,\nChikhali mein, hua racket ka ant.\n\n(Bridge)\nRaat ko chhupa, clandestine operation,\nCommercial cylinders, bhara illegal sensation.\nPimpri-Chinchwad mein

In [4]:
system_prompt = "Generate a bollywood song in hindi from this"

def generate():
  for i in range(len(result_list)-2):
    yield {"text": f"[INST]<>{system_prompt}<>{result_list[i][0].strip('English: ')}[/INST]{result_list[i][1].strip('Hinglish: ')}"}

dataset = Dataset.from_generator(generate)

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:


model_name = "meta-llama/Llama-2-7b-hf"
new_model = "llama-2-7b-article-bollywood-song"

lora_r = 64
lora_alpha = 16
lora_dropout = 0.1

use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

per_device_eval_batch_size = 4
gradient_checkpointing = True

max_seq_length = None
packing = False
device_map = {"": 0}


In [6]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [8]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    use_auth_token="hf_iyIueTWDvZTcyYmjUoTgIxeSPrCAKFoMDWkubo"
)
model.config.use_cache = False
model.config.pretraining_tp = 1


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [10]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, use_auth_token="hf_iyIueTWDvZTcyYmjUoTgIxeSPrCAKFoMDWkubo")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [12]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=15,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=10000,
    logging_steps=10,
    learning_rate=5*2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

Map:   0%|          | 0/47 [00:00<?, ? examples/s]

Step,Training Loss
10,1.391400
20,1.031500
30,0.831500
40,0.643100
50,0.552900
60,0.389300
70,0.270200
80,0.180700
90,0.143600
100,0.094900


In [13]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500)
result = pipe(f"[INST]<>{system_prompt}<>The Indian military’s modernisation budget for 2024-25 saw a meagre hike of 6.17 per cent despite the ongoing standoff with China along the Line of Actual Control in eastern Ladakh and a slew of terror attacks on security forces in Jammu and Kashmir last year....[/INST]")

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


In [14]:
result

[{'generated_text': '[INST]<>Generate a bollywood song in hindi from this<>The Indian military’s modernisation budget for 2024-25 saw a meagre hike of 6.17 per cent despite the ongoing standoff with China along the Line of Actual Control in eastern Ladakh and a slew of terror attacks on security forces in Jammu and Kashmir last year....[/INST]\n(Verse 1)\nBhool jaana hai, Indian military ka modernisation budget,\n2024-25 mein, 6.17 percent ka hike, ek naya blunder ka saath.\nLadakh ke LOC par, China ke saath haath,\nJammu aur Kashmir mein, terrorists ne kiya, security forces ke saath.\n\n(Chorus)\nMeagre hike, 6.17 percent ka, Indian military ka modernisation,\nLadakh ke LOC, Jammu aur Kashmir mein, terror attacks ka stress.\nBudget hui hai, ek naya blunder ka,\nIndian military ka modernisation, hai yeh ek tragedy ka saath.\n\n(Verse 2)\nHar kona se chal raha hai, India-China standoff,\nLine of Actual Control par, hai yeh ek track.\nSecurity forces ke safety, hai yeh ek test,\nModernis